In [2]:
# Importando bibliotecas
import re
import dask
import warnings
import numpy as np
import pandas as pd
import modin.pandas as pdm
import pyarrow.parquet as pq
import dask.dataframe as dd

from rapidfuzz import process

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option("display.max_colwidth", None)

pd.reset_option("display.max_rows")

# Verifica o uso de CPU
dask.system.cpu_count()

10

# Dados

Os dados utilizados na pesquisa e resultado de informações, de origens publica e restrita tendo como fontes o portal do [**Justiça e Números - Indicadores**](https://justica-em-numeros.cnj.jus.br/painel-estatisticas/), [**Portal da Tranparência do TJBA**](https://www.tjba.jus.br/transparencia/), [**DataJude**](https://www.cnj.jus.br/sistemas/datajud/) e PJe em parceria com o TJBA.

Os dados no Justiça e Números foram extraidos em 11/08/2025 com informações do: Painel atualizado em 25/07/2025, com dados recebidos dos tribunais e processados até 16/07/2025 (situações processuais até 30/06/2025)

Diretódio do projeto onde estão os dados:

```
dados
├── DADOS_TJBA_PG.parquet - Arquivo gerado pelo Apache Hop no cruzaemnto de dados do 1º
├── MEMBROS-DO-TJBA-E-AGENTES-PUBLICOS.csv - Arquivo do PT com lista de magistrados e lotação
└── CNJ_TJBA_JULGADOS.csv - Arquivo do JN com processos julgados TJBA já filtrados do 1º/TJBA
```

## Dados de Magistrados - Portal da Transparência
Esses dados podem demosntrar quantos magistrados exitem por vara.

In [2]:
# Inportando dados de magistrados do PT
df_magistrados = pd.read_csv('../../dados/MEMBROS_E_AGENTES_PUBLICOS_JUL2025.csv', sep=';')
df_magistrados = df_magistrados[df_magistrados['Cargo'].str.startswith('JUIZ', na=False)]

df_magistrados.head()

,Nome,Matrícula Funcional,Cargo,Função de Confiança/ Cargo em Comissão,Lotação,Ato de Provimento,Data de Publicação
7,ABRAAO BARRETO CORDEIRO,9679782,JUIZ DE DIREITO,NaN,VARA CRIM JURI EXEC PENAIS MEN,PORT S/N,16/09/2013
25,ADALBERTO LIMA BORGES FILHO,9698345,JUIZ DE DIREITO,NaN,JURISDICAO PLENA,PORT S/N,06/10/2021
69,ADERALDO DE MORAIS LEITE JUNIOR,9679871,JUIZ DE DIREITO,NaN,JURISDICAO PLENA,PORT S/N,16/09/2013
73,ADIANE JAQUELINE NEVES DA SILVA OLIVEIRA,9679812,JUIZ DE DIREITO,NaN,1ª VARA DOS SISTEMA DOS JUIZADOS ESPECIAIS,PORT S/N,16/09/2013
74,ADIDA ALVES DOS SANTOS,8098654,JUIZ DE DIREITO,NaN,2ª VARA DA INFANCIA E JUVENTUDE,PORT S/N,01/12/2004


In [18]:
df_magistrados.shape

(621, 7)

In [17]:
df_magistrados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 621 entries, 7 to 8846
Data columns (total 7 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   Nome                                    621 non-null    object
 1   Matrícula Funcional                     621 non-null    int64 
 2   Cargo                                   621 non-null    object
 3   Função de Confiança/ Cargo em Comissão  0 non-null      object
 4   Lotação                                 621 non-null    object
 5   Ato de Provimento                       621 non-null    object
 6   Data de Publicação                      621 non-null    object
dtypes: int64(1), object(6)
memory usage: 38.8+ KB


In [3]:
# Contagens de Juízes
df_magistrados.Cargo.value_counts()

Cargo
JUIZ DE DIREITO                                      544
JUIZ SUBSTITUTO                                       56
JUIZ SUBSTITUTO DE SEGUNDO GRAU - ENTRANCIA FINAL     21
Name: count, dtype: int64

In [4]:
# Pesquisa por magistrado
df_magistrados[df_magistrados['Nome'] == 'ULYSSES MAYNARD SALGADO']

,Nome,Matrícula Funcional,Cargo,Função de Confiança/ Cargo em Comissão,Lotação,Ato de Provimento,Data de Publicação
8430,ULYSSES MAYNARD SALGADO,8088900,JUIZ DE DIREITO,NaN,1A VARA FAZENDA PUBLICA,PORT S/N,23/12/2002


In [5]:
# Contagem de Lotação
df_magistrados.groupby('Lotação').agg(quant=('Lotação', 'count'))

,quant
Lotação,
1ª VARA CRIM JURI EXEC PENAIS,1
VARA EXEC. PENAIS E MEDIDAS ALTERNATIVAS,1
10ª VARA CIVEL,1
10ª VARA DE FAMILIA,1
10ª VARA DE RELACOES DE CONSUMO,1
...,...
VARA ESP CRIM INFAN JUVEN,1
VARA FEITOS RELATIVOS REL DE CONSUMO CIVEIS E COM,1
VARA FEITOS RELATIVOS REL DE CONSUMO CIVEIS E COM,1


## Dados de Processos Julgados - Indicadores CNJ
Esses dados são baijados do **Justiça em Números** mantido pelo CNJ, tendo como orijem dos dados o **DataJud**. Nesse conjunto de dados temos processos que foram julgados nos anos de 2024 e 2025.

In [2]:
# Importando dados do CNJ, processos julgados
df_cnj = pd.read_csv('../../dados/CNJ_TJBA_JULGADOS.csv', encoding='utf8', sep=';')
df_cnj.head()

,Tribunal,Grau,Nome_Orgao,UF,Municipio,Ano,Mes,Processo,Codigos_classes,Codigo_da_Ultima_classe,Nome_da_Ultima_classe,Codigos_assuntos,Data_de_Referencia,Data_do_julgamento:_Situacao:_Movimento,Data_da_decisao:_Situacao:_Movimento,Formato,Procedimento,Recurso,Codigo_Orgao,id_municipio,Polo_ativo,Polo_ativo_-_CNPJ,Polo_ativo_-_Natureza_juridica,Polo_ativo_-_CNAE,Polo_passivo,Polo_passivo_-_CNPJ,Polo_passivo_-_Natureza_juridica,Polo_passivo_-_CNAE,Poder_publico
0,TJBA,G1,VARA DE JURISDIÇÃO PLENA - ITAPICURU,BA,ITAPICURU,2024,3,8000013-59.2017.8.05.0127,{1116},1116,EXECUÇÃO FISCAL,{5952},2024-03-26,{20240326:28:459},{},Eletrônico,Execução fiscal,0,5312,2349,{MUNICIPIO DE ITAPICURU},{13647557000160},{MUNICIPIO},{ADMINISTRACAO PUBLICA EM GERAL},NaN,NaN,NaN,NaN,Poder_Publico-Polo_Ativo
1,TJBA,G1,VARA DE JURISDIÇÃO PLENA - ITAPICURU,BA,ITAPICURU,2024,4,8000014-44.2017.8.05.0127,{1116},1116,EXECUÇÃO FISCAL,{5952},2024-04-10,{20240410:28:461},{},Eletrônico,Execução fiscal,0,5312,2349,{MUNICIPIO DE ITAPICURU},{13647557000160},{MUNICIPIO},{ADMINISTRACAO PUBLICA EM GERAL},NaN,NaN,NaN,NaN,Poder_Publico-Polo_Ativo
2,TJBA,G1,VARA DE JURISDIÇÃO PLENA - ITAPICURU,BA,ITAPICURU,2024,3,8000894-36.2017.8.05.0127,{1116},1116,EXECUÇÃO FISCAL,{5952},2024-03-26,{20240326:28:459},{},Eletrônico,Execução fiscal,0,5312,2349,{MUNICIPIO DE ITAPICURU},{13647557000160},{MUNICIPIO},{ADMINISTRACAO PUBLICA EM GERAL},NaN,NaN,NaN,NaN,Poder_Publico-Polo_Ativo
3,TJBA,G1,VARA DE JURISDIÇÃO PLENA - ITAPICURU,BA,ITAPICURU,2024,3,8000012-74.2017.8.05.0127,{1116},1116,EXECUÇÃO FISCAL,{5952},2024-03-26,{20240326:28:459},{},Eletrônico,Execução fiscal,0,5312,2349,{MUNICIPIO DE ITAPICURU},{13647557000160},{MUNICIPIO},{ADMINISTRACAO PUBLICA EM GERAL},NaN,NaN,NaN,NaN,Poder_Publico-Polo_Ativo
4,TJBA,G1,VARA DE JURISDIÇÃO PLENA - ITAPICURU,BA,ITAPICURU,2024,3,0000753-32.2012.8.05.0127,{1116},1116,EXECUÇÃO FISCAL,{10887},2024-03-26,{20240326:28:459},{},Eletrônico,Execução fiscal,0,5312,2349,{INSTITUTO NACIONAL DE METROLOGIA QUALIDADE E TECNOLOGIA - INMETRO.},{00662270000168},{AUTARQUIA FEDERAL},{ADMINISTRACAO PUBLICA EM GERAL},{G D DE SOUZA DE ITAPICURU},{07593292000162},{EMPRESARIO (INDIVIDUAL)},{FABRICACAO DE PRODUTOS DE PADARIA E CONFEITARIA COM PREDOMINANCIA DE PRODUCAO PROPRIA},Poder_Publico-Polo_Ativo


In [3]:
df_cnj.shape

(1440265, 29)

In [4]:
# Processos que tiveream mais de uma decisão
df_cnj['Processo'].value_counts()[lambda x: x > 1]

Processo
0700281-17.2021.8.05.0113    12
8005064-54.2021.8.05.0113     7
0001917-26.2002.8.05.0113     7
0011275-05.2008.8.05.0113     7
8005315-83.2022.8.05.0001     6
                             ..
8123251-66.2021.8.05.0001     2
8001617-40.2018.8.05.0153     2
8083505-26.2023.8.05.0001     2
8013338-63.2021.8.05.0256     2
8004735-32.2020.8.05.0063     2
Name: count, Length: 42322, dtype: int64

## Cruzamento dos Dados

Nessa fase precisamos cruzar alguns dados, apesar de termos fontes como **CNJ(DataJUD)**, algumas informações são mais específicas e precisamos cruzar essas informações com o TJBA. Nessa primeira etapa devido ao volume e acomplexidade, foi utilizado a ferramenta de ETL [**Apache Hop**](https://hop.apache.org/), para o cruzamento, análises e um tratamento inicial. O artefado final gerado nessa etapa será um arquivo parquet, que usaremos como principal fonte de dados para o modelo.

<p align="center">
    <img width="850" height="250" src="../../img/001.png">
</p>

## Tratamento dos Dados

Aqui vamos trabalhar com os dados gerado na etapa de cruzemnto, realizando alguns tratamentos e adequando os dados;

*	Selecionar as melhores features (colunas).
*	Remover features (colunas) dos df's.
*	Verificar valores ausentes que possam impactar. 
*	Verificar qual fonte dos dados relacionado a sistemas vamos utilizar.
*	Trabalhar na questão dos magistrados.
*	Etc...






### Tratando Dados do TJBA

In [3]:
# Verificando eschema do arquivo parquet gerado
table = pq.read_table("../../dados/DADOS_TJBA_PG.parquet")
print(table.schema)

pje_processo: string
vm_processo: string
vm_dt_recebimento: timestamp[ms, tz=UTC]
vm_dt_movimento: timestamp[ms, tz=UTC]
vm_cod_cnj_comarca: int64
vm_mapa_comarca: int64
vm_cod_comarca: int64
vm_comarca: string
pje_id_jurisdicao: int64
pje_ds_jurisdicao: string
vm_mapa_vara: int64
vm_cod_vara: int64
vm_cod_cnj_vara: int64
pje_id_orgao_julgador: int64
pje_nr_vara: int64
vm_vara: string
pje_ds_orgao_julgador: string
pje_cd_classe_judicial: string
vm_cod_cls_cnj: int64
vm_classe: string
pje_ds_classe_judicial: string
pje_ds_natureza: string
pje_vl_peso: int64
pje_cd_assunto_principal: string
pje_ds_assunto_principal: string
vm_cod_mov_cnj: int64
vm_movimento: string
pje_id_competencia: int64
pje_ds_competencia: string
vm_competencia: string
vm_localizacao: string
vm_tp_proc: string
pje_id_orgao_julgador_cargo: int64
pje_ds_orgao_julgador_cargo: string
vm_cod_cnj_magistrado: int64
vm_mapa_magistrado: int64
vm_cod_magistrado: string
vm_magistrado: string
vm_consulta: string
vm_situacao: str

In [3]:
# Importando arquivo parquet gerado pelo Apache Hop no Pandas
df_tjba = pd.read_parquet('../../dados/DADOS_TJBA_PG.parquet', engine="pyarrow", use_pandas_metadata=False)

# Importando arquivo parquet gerado pelo Apache Hop no Dask
#df_tjba = dd.read_parquet('../../dados/DADOS_TJBA_PG.parquet', engine="pyarrow", use_pandas_metadata=False)

# Conver timestamp para datetime
#df["cnj_dt_referencia"] = df["cnj_dt_referencia"].dt.tz_convert(None)
#df["vm_dt_recebimento"] = df["vm_dt_recebimento"].dt.tz_convert(None)
#df["vm_dt_movimento"] = df["vm_dt_movimento"].dt.tz_convert(None)

In [10]:
df_tjba.shape

(11715246, 50)

In [11]:
df_tjba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11715246 entries, 0 to 11715245
Data columns (total 50 columns):
 #   Column                       Dtype              
---  ------                       -----              
 0   pje_processo                 object             
 1   vm_processo                  object             
 2   vm_dt_recebimento            datetime64[ms, UTC]
 3   vm_dt_movimento              datetime64[ms, UTC]
 4   vm_cod_cnj_comarca           int64              
 5   vm_mapa_comarca              int64              
 6   vm_cod_comarca               int64              
 7   vm_comarca                   object             
 8   pje_id_jurisdicao            float64            
 9   pje_ds_jurisdicao            object             
 10  vm_mapa_vara                 int64              
 11  vm_cod_vara                  int64              
 12  vm_cod_cnj_vara              int64              
 13  pje_id_orgao_julgador        float64            
 14  pje_nr_vara     

In [4]:
df_tjba.head()

,pje_processo,vm_processo,vm_dt_recebimento,vm_dt_movimento,vm_cod_cnj_comarca,vm_mapa_comarca,vm_cod_comarca,vm_comarca,pje_id_jurisdicao,pje_ds_jurisdicao,vm_mapa_vara,vm_cod_vara,vm_cod_cnj_vara,pje_id_orgao_julgador,pje_nr_vara,vm_vara,pje_ds_orgao_julgador,pje_cd_classe_judicial,vm_cod_cls_cnj,vm_classe,pje_ds_classe_judicial,pje_ds_natureza,pje_vl_peso,pje_cd_assunto_principal,pje_ds_assunto_principal,vm_cod_mov_cnj,vm_movimento,pje_id_competencia,pje_ds_competencia,vm_competencia,vm_localizacao,vm_tp_proc,pje_id_orgao_julgador_cargo,pje_ds_orgao_julgador_cargo,vm_cod_cnj_magistrado,vm_mapa_magistrado,vm_cod_magistrado,vm_magistrado,vm_consulta,vm_situacao,vm_entrancia,vm_compl_mov,vm_tempo_tramitacao,pje_vl_peso_processual,vm_prioridade,vm_ds_prioridade,pje_vl_peso_prioridade,vm_concluso,vm_sistema,vm_num_ativo
0,8003619-75.2022.8.05.0271,8003619-75.2022.8.05.0271,2022-09-19 14:02:01+00:00,2024-08-26 12:47:34+00:00,5359,403,1255,VALENCA,255.0,VALENÇA,1642,11534,50076,534.0,1248.0,2ª VARA DE FEITOS DE REL DE CONS. CÍVEL E COMERCIAIS,2ª V DOS FEITOS DE REL. DE CONS. CÍVEIS COM. FAZ. PUB. E ACID. TRAB. DE VALENÇA,65,65,ACAO CIVIL PUBLICA,AÇÃO CIVIL PÚBLICA,CONHECIMENTO,2.0,10113,Flora,11010,PROFERIDO DESPACHO DE MERO EXPEDIENTE,9.0,Fazenda Pública - Atos Administrativos,FAZENDA PUBLICA ATOS ADMINISTRATIVOS,DIREÇÃO DE SECRETARIA,DIGITAL,1125.0,Juiz de Direito Titular,136647.0,378.0,1153579,LEONARDO RULIAN CUSTODIO,DESPACHO,EM ANDAMENTO,FINAL,,0.0,4.0,N,None,0.0,0,PJEPG,1
1,8003068-15.2022.8.05.0039,8003068-15.2022.8.05.0039,2022-02-06 18:58:24+00:00,2024-09-19 15:03:05+00:00,903,68,115,CAMACARI,15.0,CAMAÇARI,2004,11453,79440,453.0,45.0,2ª VARA DA FAZENDA PÚBLICA,2ª V DE FAZENDA PUBLICA DE CAMAÇARI,1116,1116,EXECUCAO FISCAL,EXECUÇÃO FISCAL,EXECUçãO,2.0,6017,Dívida Ativa (Execução Fiscal),461,EXTINTO O PROCESSO POR AUSENCIA DAS CONDICOES DA ACAO,6.0,Fazenda Pública - Tributos,FAZENDA PUBLICA TRIBUTOS,DIREÇÃO DE SECRETARIA,DIGITAL,935.0,JUIZ DE DIREITO,4690.0,166.0,1858441,DANIEL LIMA FALCAO,JULGAMENTO,BAIXADO,FINAL,SEM MERITO,0.0,4.0,N,None,0.0,0,PJEPG,1
2,0088391-11.2003.8.05.0001,0088391-11.2003.8.05.0001,2003-07-18 13:10:09+00:00,2025-07-18 22:24:10+00:00,4278,334,18,SALVADOR,8.0,SALVADOR - REGIÃO METROPOLITANA,4202,111452,90461,1452.0,15.0,15ª VARA DA FAZENDA PÚBLICA,15ª V DA FAZENDA PÚBLICA DE SALVADOR,1116,1116,EXECUCAO FISCAL,EXECUÇÃO FISCAL,EXECUçãO,2.0,12989,Execução Fiscal,11010,PROFERIDO DESPACHO DE MERO EXPEDIENTE,9.0,Fazenda Pública - Atos Administrativos,FAZENDA PUBLICA ATOS ADMINISTRATIVOS,DIREÇÃO DE SECRETARIA,DIGITAL,2853.0,Juiz de Direito Titular,4720.0,161.0,12898787,CRISTIANE MENEZES SANTOS BARRETO,DESPACHO,EM ANDAMENTO,FINAL,,0.0,4.0,N,None,0.0,0,PJEPG,1
3,0000087-73.2019.8.05.0066,0000087-73.2019.8.05.0066,2019-03-18 18:20:00+00:00,2025-07-09 17:53:34+00:00,1337,103,1165,CONDEUBA,165.0,CONDEÚBA,273,11754,5130,754.0,314.0,VARA JURISDIÇÃO PLENA,VARA CRIMINAL DE CONDEÚBA,283,283,ACAO PENAL PROCEDIMENTO ORDINARIO,AÇÃO PENAL - PROCEDIMENTO ORDINÁRIO,CONHECIMENTO,2.0,11417,Estupro de vulnerável,11010,PROFERIDO DESPACHO DE MERO EXPEDIENTE,53.0,CRIME,CRIME,DIREÇÃO DE SECRETARIA,DIGITAL,1740.0,JUIZ SUBSTITUTO,150543.0,1492.0,13523090,CARLOS TIAGO SILVA ADAES,DESPACHO,EM ANDAMENTO,INICIAL,,0.0,4.0,N,None,0.0,0,PJEPG,1
4,8055437-37.2021.8.05.0001,8055437-37.2021.8.05.0001,2021-05-28 13:47:59+00:00,2025-05-30 17:27:43+00:00,4278,334,18,SALVADOR,8.0,SALVADOR - REGIÃO METROPOLITANA,716,11553,17167,553.0,923.0,6ª VARA CÍVEL E COMERCIAL,6ª V CÍVEL E COMERCIAL DE SALVADOR,7,7,PROCEDIMENTO COMUM CIVEL,PROCEDIMENTO COMUM CÍVEL,CONHECIMENTO,2.0,9597,Seguro,11010,PROFERIDO DESPACHO DE MERO EXPEDIENTE,12.0,Cível,CIVEL,DIREÇÃO DE SECRETARIA,DIGITAL,1165.0,Juiz de Direito Titular,0.0,750.0,1133966,CARLOS CARVALHO RAMOS DE CERQUEIRA JUNIOR,DESPACHO,EM ANDAMENTO,FINAL,,0.0,4.0,N,None,0.0,0,PJEPG,1


In [5]:
# Removendo fetures
df_tjba.drop([
    'pje_processo', 'vm_dt_recebimento', 'vm_cod_cnj_comarca', 'vm_mapa_comarca', 'pje_id_jurisdicao', 
    'pje_ds_jurisdicao', 'vm_mapa_vara', 'vm_cod_cnj_vara', 'pje_id_orgao_julgador', 'pje_nr_vara', 
    'pje_ds_orgao_julgador', 'pje_cd_classe_judicial', 'pje_ds_classe_judicial', 'pje_vl_peso', 
    'pje_cd_assunto_principal', 'pje_ds_assunto_principal', 'vm_cod_mov_cnj', 'vm_movimento',
    'pje_ds_competencia', 'vm_localizacao', 'vm_tp_proc', 'pje_id_orgao_julgador_cargo', 'pje_ds_orgao_julgador_cargo',
    'vm_cod_cnj_magistrado', 'vm_cod_magistrado', 'vm_entrancia', 'vm_compl_mov', 'vm_tempo_tramitacao', 
    'pje_vl_peso_prioridade', 'vm_concluso', 'vm_num_ativo'
], axis=1, inplace=True)

In [6]:
# Algumas colunas tem valores None indicando ausência de dados, mudando para NaN
#df_tjba['vm_magistrado'] = df_tjba['vm_magistrado'].apply(lambda x: np.nan if x is None else x)

# Colocando dados em letras maiúsculas
#df_tjba['pje_ds_assunto_principal'] = df_tjba['pje_ds_assunto_principal'].str.upper()
#df_tjba['pje_ds_orgao_julgador_cargo'] = df_tjba['pje_ds_orgao_julgador_cargo'].str.upper()
df_tjba['pje_ds_natureza'] = df_tjba['pje_ds_natureza'].str.upper()
df_tjba['vm_ds_prioridade'] = df_tjba['vm_ds_prioridade'].str.upper()

# Retirando o timestamp
#df_tjba['vm_dt_recebimento'] = df_tjba['vm_dt_recebimento'].dt.date
#df_tjba['vm_dt_recebimento'] = pd.to_datetime(df_tjba['vm_dt_recebimento']) # Erro 1001-10-30, at position 93295
df_tjba['vm_dt_movimento'] = df_tjba['vm_dt_movimento'].dt.date
df_tjba['vm_dt_movimento'] = pd.to_datetime(df_tjba['vm_dt_movimento'])

In [7]:
# Percorre todas as colunas do df_tjba do tipo object, retirando espaços laterais e mais de um espaços entre palavras
for col in df_tjba.select_dtypes(include=['object']).columns:
    df_tjba[col] = (
        df_tjba[col].str.strip().str.replace(r'\s+', ' ', regex=True)
	)

In [8]:
df_tjba.head()

,vm_processo,vm_dt_movimento,vm_cod_comarca,vm_comarca,vm_cod_vara,vm_vara,vm_cod_cls_cnj,vm_classe,pje_ds_natureza,pje_id_competencia,vm_competencia,vm_mapa_magistrado,vm_magistrado,vm_consulta,vm_situacao,pje_vl_peso_processual,vm_prioridade,vm_ds_prioridade,vm_sistema
0,8003619-75.2022.8.05.0271,2024-08-26,1255,VALENCA,11534,2ª VARA DE FEITOS DE REL DE CONS. CÍVEL E COMERCIAIS,65,ACAO CIVIL PUBLICA,CONHECIMENTO,9.0,FAZENDA PUBLICA ATOS ADMINISTRATIVOS,378.0,LEONARDO RULIAN CUSTODIO,DESPACHO,EM ANDAMENTO,4.0,N,None,PJEPG
1,8003068-15.2022.8.05.0039,2024-09-19,115,CAMACARI,11453,2ª VARA DA FAZENDA PÚBLICA,1116,EXECUCAO FISCAL,EXECUÇÃO,6.0,FAZENDA PUBLICA TRIBUTOS,166.0,DANIEL LIMA FALCAO,JULGAMENTO,BAIXADO,4.0,N,None,PJEPG
2,0088391-11.2003.8.05.0001,2025-07-18,18,SALVADOR,111452,15ª VARA DA FAZENDA PÚBLICA,1116,EXECUCAO FISCAL,EXECUÇÃO,9.0,FAZENDA PUBLICA ATOS ADMINISTRATIVOS,161.0,CRISTIANE MENEZES SANTOS BARRETO,DESPACHO,EM ANDAMENTO,4.0,N,None,PJEPG
3,0000087-73.2019.8.05.0066,2025-07-09,1165,CONDEUBA,11754,VARA JURISDIÇÃO PLENA,283,ACAO PENAL PROCEDIMENTO ORDINARIO,CONHECIMENTO,53.0,CRIME,1492.0,CARLOS TIAGO SILVA ADAES,DESPACHO,EM ANDAMENTO,4.0,N,None,PJEPG
4,8055437-37.2021.8.05.0001,2025-05-30,18,SALVADOR,11553,6ª VARA CÍVEL E COMERCIAL,7,PROCEDIMENTO COMUM CIVEL,CONHECIMENTO,12.0,CIVEL,750.0,CARLOS CARVALHO RAMOS DE CERQUEIRA JUNIOR,DESPACHO,EM ANDAMENTO,4.0,N,None,PJEPG


In [18]:
df_tjba.shape

(11715246, 19)

In [9]:
# Selecionando dados de 2024 e do PJE e registros somente com magistrados
df_tjba_fil = df_tjba[(df_tjba['vm_dt_movimento'].dt.year == 2024) & 
                    (df_tjba['vm_sistema'] == 'PJEPG') & 
                    (df_tjba['vm_magistrado'].isna() == False)]

# Convertendo float para int
df_tjba_fil['pje_vl_peso_processual'] = df_tjba_fil['pje_vl_peso_processual'].astype('Int64')
#df_tjba_fil['pje_vl_peso_prioridade'] = df_tjba_fil['pje_vl_peso_prioridade'].astype('Int64')
df_tjba_fil['vm_mapa_magistrado'] = df_tjba_fil['vm_mapa_magistrado'].astype('Int64')

# Criando campo para identificação dos magistrados
df_tjba_fil['juiz_str'] = 'JUIZ_' + df_tjba_fil['vm_mapa_magistrado'].astype(str)
df_tjba_fil['vara_str'] = 'VARA_' + df_tjba_fil['vm_cod_vara'].astype(str)

In [20]:
df_tjba_fil.shape

(4225839, 21)

In [ ]:
#nomes_magistrados = df_tjba['vm_magistrado'].dropna().unique()
#
#df_magistrados['vm_nome'] = [
#    next((nome for nome in nomes_magistrados if str(nome).startswith(str(cadastro))), None) 
#    for cadastro in df_magistrados['Nome']
#]

In [ ]:
## Agora com os nomes ajustados podemos colocando a matrícula dos magistrados 
## que temos no cadastro no df principal
#df_tjba = df_tjba.merge(
#    df_magistrados[['Nome', 'Matrícula Funcional']], 
#    left_on='vm_magistrado', 
#    right_on='Nome', 
#    how='left'
#).rename(columns={'Matrícula Funcional': 'pt_matricula'})
#
## Removendo coluna Nome
#df_tjba.drop('Nome', axis=1, inplace=True)
#
## Pegando lista de colunas
#cols = list(df_tjba.columns)
#
## Removendo coluna
#cols.remove('pt_matricula')
#
## Achando posição, onde adicionar coluna
#pos = cols.index('vm_mapa_magistrado') + 1
#
## Adicionado coluna na posição escolhida
#cols.insert(pos, 'pt_matricula')
#
## Reordenado df
#df_tjba = df_tjba[cols]

### Tratando Dados do Portal da Transparência

In [10]:
# Alguns nomes do df de cadastro (PT) não batem com o do TJBA, ajustando o cadastro por similaridade
df_magistrados['vm_nome'] = np.nan
nomes_mov = df_tjba['vm_magistrado'].dropna().unique()

for i, row in df_magistrados.iterrows():
    nome_cadastro = row['Nome']
    match, score, idx = process.extractOne(nome_cadastro, nomes_mov)
    
    if score > 85:  # limiar de confiança
        df_magistrados.at[i, 'vm_nome'] = match

In [11]:
df_magistrados.head()

,Nome,Matrícula Funcional,Cargo,Função de Confiança/ Cargo em Comissão,Lotação,Ato de Provimento,Data de Publicação,vm_nome
7,ABRAAO BARRETO CORDEIRO,9679782,JUIZ DE DIREITO,NaN,VARA CRIM JURI EXEC PENAIS MEN,PORT S/N,16/09/2013,ABRAAO BARRETO CORDEIRO
25,ADALBERTO LIMA BORGES FILHO,9698345,JUIZ DE DIREITO,NaN,JURISDICAO PLENA,PORT S/N,06/10/2021,ADALBERTO LIMA BORGES FILHO
69,ADERALDO DE MORAIS LEITE JUNIOR,9679871,JUIZ DE DIREITO,NaN,JURISDICAO PLENA,PORT S/N,16/09/2013,ADERALDO DE MORAIS LEITE JUNIOR
73,ADIANE JAQUELINE NEVES DA SILVA OLIVEIRA,9679812,JUIZ DE DIREITO,NaN,1ª VARA DOS SISTEMA DOS JUIZADOS ESPECIAIS,PORT S/N,16/09/2013,ADIANE JAQUELINE NEVES DA SILVA
74,ADIDA ALVES DOS SANTOS,8098654,JUIZ DE DIREITO,NaN,2ª VARA DA INFANCIA E JUVENTUDE,PORT S/N,01/12/2004,ADIDA ALVES DOS SANTOS


In [12]:
# Verificando se todos os nomes foram preenchidos
df_magistrados[df_magistrados['vm_nome'].isna()]

,Nome,Matrícula Funcional,Cargo,Função de Confiança/ Cargo em Comissão,Lotação,Ato de Provimento,Data de Publicação,vm_nome


In [ ]:
#
#for i, row, in df_magistrados.iterrows():
#    magistrado = row['vm_nome']
#    dados = df_tjba.loc[df_tjba['vm_magistrado'] == magistrado, ['vm_cod_comarca', 'vm_comarca', 'vm_cod_vara']]
#    
#    if not dados.empty:
#        df_magistrados.at[i, 'vm_cod_comarca'] = dados.iloc[0]['vm_cod_comarca']
#        df_magistrados.at[i, 'vm_comarca'] = dados.iloc[0]['vm_comarca']
#        df_magistrados.at[i, 'vm_cod_vara'] = dados.iloc[0]['vm_cod_vara']

## Análise Exploratória - Conjunto de Dados Final

In [21]:
# Quantidade de registro por sistema df TJBA
df_tjba['vm_sistema'].value_counts()

vm_sistema
PJEPG     11556834
SEEU        158333
SAJPG           71
SAIPRO           8
Name: count, dtype: int64

In [22]:
# Verificando pessos dos processos df TJBA
df_tjba['pje_vl_peso_processual'].value_counts()

pje_vl_peso_processual
4.0    11537248
2.0       19571
1.0          15
Name: count, dtype: int64

In [23]:
# Verificando pessos dos processos df filtrado
df_tjba_fil['pje_vl_peso_processual'].value_counts()

pje_vl_peso_processual
4    4218260
2       7574
1          5
Name: count, dtype: Int64

In [25]:
# Verifica quantidade de processos por consulta
df_tjba_fil['vm_consulta'].value_counts()

vm_consulta
DESPACHO      2000967
JULGAMENTO    1116059
DECISAO       1108813
Name: count, dtype: int64

In [26]:
# Verificando existencia de valores ausentes
df_tjba_fil[df_tjba_fil['vm_mapa_magistrado'].isna()]
#df_tjba_fil[df_tjba_fil['pje_vl_peso_processual'].isna()]

,vm_processo,vm_dt_movimento,vm_cod_comarca,vm_comarca,vm_cod_vara,vm_vara,vm_cod_cls_cnj,vm_classe,pje_ds_natureza,pje_id_competencia,vm_competencia,vm_mapa_magistrado,vm_magistrado,vm_consulta,vm_situacao,pje_vl_peso_processual,vm_prioridade,vm_ds_prioridade,vm_sistema,juiz_str,vara_str


In [27]:
# Verifica tipo de consultas
print('Tipos de consulta: ', df_tjba_fil['vm_consulta'].unique())
print('Quantidade de magistrados: ',df_tjba_fil['vm_magistrado'].nunique())
print('Quantidade de registros', df_tjba_fil.shape)
print('Quantidade de processos', df_tjba_fil['vm_processo'].nunique())

Tipos de consulta:  ['DESPACHO' 'JULGAMENTO' 'DECISAO']
Quantidade de magistrados:  624
Quantidade de registros (4225839, 21)
Quantidade de processos 2706715


## Feature Enginer - Eriquecimento dos Dados

In [10]:
# Gerando df com informação de magistrados por Comarca
df_comarca = (
    df_tjba_fil.groupby(['vm_cod_comarca', 'vm_comarca'])['vm_magistrado'].nunique().reset_index(name='Qtd. Magistrado')
).rename(columns={
    'vm_cod_comarca': 'Cod. Comarca',
    'vm_comarca': 'Comarca'
})

df_comarca


,Cod. Comarca,Comarca,Qtd. Magistrado
0,18,SALVADOR,614
1,110,PORTO SEGURO,198
2,111,EUNAPOLIS,157
3,112,ITANHEM,20
4,114,BARRA,44
...,...,...,...
198,1253,ITAPETINGA,138
199,1254,ITABUNA,203
200,1255,VALENCA,147
201,1256,IPIAU,58


In [ ]:
# Gerando df com informação de magistrados por Vara
df_vara = (
    df_tjba_fil.groupby(['vm_cod_comarca', 'vm_comarca', 'vara_str', 'vm_vara'])['vm_magistrado']
    .nunique().reset_index(name='Qtd. Total')
).rename(columns={
    'vm_cod_comarca': 'Cod. Comarca',
    'vm_comarca': 'Comarca',
    'vara_str': 'Cod. Vara',
    'vm_vara': 'Vara'
})

# Primeiro trimestre
df_vara['T1'] = df_vara['Cod. Vara'].map(
    df_tjba_fil[df_tjba_fil['vm_dt_movimento'].dt.quarter == 1]
    .groupby('vara_str')['juiz_str'].nunique()
).fillna(0).astype(int)

# Segundo trimestre
df_vara['T2'] = df_vara['Cod. Vara'].map(
    df_tjba_fil[df_tjba_fil['vm_dt_movimento'].dt.quarter == 2]
    .groupby('vara_str')['juiz_str'].nunique()
).fillna(0).astype(int)

# Terceiro trimestre
df_vara['T3'] = df_vara['Cod. Vara'].map(
    df_tjba_fil[df_tjba_fil['vm_dt_movimento'].dt.quarter == 3]
    .groupby('vara_str')['juiz_str'].nunique()
).fillna(0).astype(int)

# Quarto trimestre
df_vara['T4'] = df_vara['Cod. Vara'].map(
    df_tjba_fil[df_tjba_fil['vm_dt_movimento'].dt.quarter == 4]
    .groupby('vara_str')['juiz_str'].nunique()
).fillna(0).astype(int)

df_vara

,Cod. Comarca,Comarca,Cod. Vara,Vara,Qtd. Total,T1,T2,T3,T4
0,18,SALVADOR,VARA_111233,[CEJUSC PRÉ-PROCESSUAL] - BAIRRO DA PAZ,2,2,1,1,1
1,18,SALVADOR,VARA_111234,[CEJUSC] BARRIS,1,0,0,1,1
2,18,SALVADOR,VARA_111235,[CEJUSC PRÉ-PROCESSUAL] - BOCA DO RIO,1,1,0,0,0
3,18,SALVADOR,VARA_111236,[CEJUSC PRE-PROCESSUAL] - CABULA,3,2,1,1,2
4,18,SALVADOR,VARA_111237,[CEJUSC PRE-PROCESSUAL] - CAJAZEIRAS 10 PREFEITURA,2,2,1,1,1
...,...,...,...,...,...,...,...,...,...
845,1257,TEIXEIRA DE FREITAS,VARA_11543,[CEJUSC] TEIXEIRA DE FREITAS,2,1,1,0,1
846,1257,TEIXEIRA DE FREITAS,VARA_11834,1ª VARA CRIMINAL,45,11,10,12,17
847,1257,TEIXEIRA DE FREITAS,VARA_11835,"2ª VARA CRIME, PRIVATIVA DE VIOLÊNCIA DOMÉSTICA",36,15,7,9,16
848,1257,TEIXEIRA DE FREITAS,VARA_11836,1ª VARA DE EXECUÇÕES PENAIS E JÚRI,13,6,6,2,8


In [40]:
# Verificando a quantidade varas que cada juiz atua
df_juiz_max_vara = df_tjba_fil.groupby('juiz_str')['vara_str'].nunique().reset_index().sort_values('vara_str', ascending=False)
df_juiz_max_vara.columns = ['Cod. Juiz', 'Qtd. Vara']

# Primeiro trimestre
df_juiz_max_vara['T1'] = df_juiz_max_vara['Cod. Juiz'].map(
    df_tjba_fil[df_tjba_fil['vm_dt_movimento'].dt.quarter == 1]
    .groupby('juiz_str')['vara_str'].nunique()
).fillna(0).astype(int)

# Segundo trimestre
df_juiz_max_vara['T2'] = df_juiz_max_vara['Cod. Juiz'].map(
    df_tjba_fil[df_tjba_fil['vm_dt_movimento'].dt.quarter == 2]
    .groupby('juiz_str')['vara_str'].nunique()
).fillna(0).astype(int)

# Terceiro trimestre
df_juiz_max_vara['T3'] = df_juiz_max_vara['Cod. Juiz'].map(
    df_tjba_fil[df_tjba_fil['vm_dt_movimento'].dt.quarter == 3]
    .groupby('juiz_str')['vara_str'].nunique()
).fillna(0).astype(int)

# Quarto trimestre
df_juiz_max_vara['T4'] = df_juiz_max_vara['Cod. Juiz'].map(
    df_tjba_fil[df_tjba_fil['vm_dt_movimento'].dt.quarter == 4]
    .groupby('juiz_str')['vara_str'].nunique()
).fillna(0).astype(int)

df_juiz_max_vara

,Cod. Juiz,Qtd. Vara,T1,T2,T3,T4
575,JUIZ_669,182,88,109,109,62
603,JUIZ_818,166,49,39,96,54
21,JUIZ_123,121,64,67,65,69
249,JUIZ_231,121,66,76,71,51
465,JUIZ_525,111,68,60,63,65
...,...,...,...,...,...,...
68,JUIZ_14,4,4,0,0,0
225,JUIZ_195,3,3,0,0,0
166,JUIZ_1627,3,3,0,0,0
153,JUIZ_1614,2,2,0,0,0


In [12]:
# Gerando df com produtividade ponderada dos Juízes (P_i) - usando transform
# Criar um registro para cada juiz mostrando a soma dos "pesos" dos processos que ele julgou em um período (trimestral)
P_i = df_tjba_fil[['juiz_str']].drop_duplicates().reset_index(drop=True)

# Criando produtivedade ponderada para 1º trimestre
P_i['T1'] = P_i['juiz_str'].map(
    df_tjba_fil[df_tjba_fil['vm_dt_movimento'].dt.quarter == 1]
    .groupby('juiz_str')['pje_vl_peso_processual'].sum()
).fillna(0)

# Criando produtivedade ponderada para 2º trimestre
P_i['T2'] = P_i['juiz_str'].map(
    df_tjba_fil[df_tjba_fil['vm_dt_movimento'].dt.quarter == 2]
    .groupby('juiz_str')['pje_vl_peso_processual'].sum()
).fillna(0)

# Criando produtivedade ponderada para 3º trimestre
P_i['T3'] = P_i['juiz_str'].map(
    df_tjba_fil[df_tjba_fil['vm_dt_movimento'].dt.quarter == 3]
    .groupby('juiz_str')['pje_vl_peso_processual'].sum()
).fillna(0)

# Criando produtivedade ponderada para 4º trimestre
P_i['T4'] = P_i['juiz_str'].map(
    df_tjba_fil[df_tjba_fil['vm_dt_movimento'].dt.quarter == 4]
    .groupby('juiz_str')['pje_vl_peso_processual'].sum()
).fillna(0)

P_i

,juiz_str,T1,T2,T3,T4
0,JUIZ_378,19464,17628,22728,16204
1,JUIZ_166,39804,76880,124576,105942
2,JUIZ_649,12116,18222,20526,16826
3,JUIZ_599,6710,7284,6096,4770
4,JUIZ_1638,5188,8448,8650,6912
...,...,...,...,...,...
619,JUIZ_337,4,0,4,0
620,JUIZ_224,20,0,24,0
621,JUIZ_793,32,4,8,0
622,JUIZ_502,0,0,0,56


In [13]:
# Verificando o caso do magistrado (JUIZ_337) com baixa produtividade
df_tjba_fil[df_tjba_fil['juiz_str'] == 'JUIZ_337']

,vm_processo,vm_dt_movimento,vm_cod_comarca,vm_comarca,vm_cod_vara,vm_vara,vm_cod_cls_cnj,vm_classe,pje_ds_natureza,pje_id_competencia,vm_competencia,vm_mapa_magistrado,vm_magistrado,vm_consulta,vm_situacao,pje_vl_peso_processual,vm_prioridade,vm_ds_prioridade,vm_sistema,juiz_str,vara_str
3290908,8003313-77.2021.8.05.0001,2024-03-23,18,SALVADOR,1115,13ª VARA DA FAZENDA PÚBLICA,1116,EXECUCAO FISCAL,EXECUÇÃO,46.0,FAZENDA PUBLICA EXECUCAO MUNICIPIO,337,JOSE LUIZ PESSOA CARDOSO,DECISAO,SUSPENSO,4,N,None,PJEPG,JUIZ_337,VARA_1115
5525618,8116005-19.2021.8.05.0001,2024-08-29,18,SALVADOR,1115,13ª VARA DA FAZENDA PÚBLICA,1116,EXECUCAO FISCAL,EXECUÇÃO,46.0,FAZENDA PUBLICA EXECUCAO MUNICIPIO,337,JOSE LUIZ PESSOA CARDOSO,DECISAO,SUSPENSO,4,N,None,PJEPG,JUIZ_337,VARA_1115


In [14]:
# Cria um df Criar com indicador mostrando em quais naturezas de vara cada juiz tem experiência (E_ik)
E_ik = df_tjba_fil[['juiz_str']].drop_duplicates().reset_index(drop=True)
naturezas = df_tjba_fil[['vm_competencia']].drop_duplicates().reset_index(drop=True)
E_ik = E_ik.merge(naturezas, how='cross')

E_ik = E_ik.merge(
    df_tjba_fil[['juiz_str', 'vm_competencia']].drop_duplicates(),
    on=['juiz_str', 'vm_competencia'],
    how='left',
    indicator=True
)

E_ik['flag'] = (E_ik['_merge'] == 'both').astype(int)
E_ik = E_ik.drop(columns=['_merge'], axis=1)

E_ik

,juiz_str,vm_competencia,flag
0,JUIZ_378,FAZENDA PUBLICA ATOS ADMINISTRATIVOS,1
1,JUIZ_378,FAZENDA PUBLICA TRIBUTOS,1
2,JUIZ_378,FAMILIA,1
3,JUIZ_378,RELACOES DE CONSUMO,1
4,JUIZ_378,ACIDENTES DO TRABALHO,1
...,...,...,...
63643,JUIZ_205,EXECUCOES RECESSO SALVADOR,0
63644,JUIZ_205,[CEJUSC PREPROCESSUAL] ZONA IV,0
63645,JUIZ_205,[CEJUSC PREPROCESSUAL FAMILIA] ZONA 18,0
63646,JUIZ_205,CRIMINAL RECESSO SALVADOR,0


In [15]:
# Gerando df com capacidade ponderada das Varas por período (C_j)
# Criar registro para cada vara mostrando soma dos "pesos" que ela consegue processar
C_j = df_tjba_fil[['vara_str', 'vm_vara']].drop_duplicates().reset_index(drop=True)

# Criando capacidade ponderada para 1º trimestre
C_j['T1'] = C_j['vara_str'].map(
    df_tjba_fil[df_tjba_fil['vm_dt_movimento'].dt.quarter == 1]
    .groupby('vara_str')['pje_vl_peso_processual'].sum()
).fillna(0)

# Criando capacidade ponderada para 2º trimestre
C_j['T2'] = C_j['vara_str'].map(
    df_tjba_fil[df_tjba_fil['vm_dt_movimento'].dt.quarter == 2]
    .groupby('vara_str')['pje_vl_peso_processual'].sum()
).fillna(0)

# Criando capacidade ponderada para 3º trimestre
C_j['T3'] = C_j['vara_str'].map(
    df_tjba_fil[df_tjba_fil['vm_dt_movimento'].dt.quarter == 3]
    .groupby('vara_str')['pje_vl_peso_processual'].sum()
).fillna(0)

# Criando capacidade ponderada para 4º trimestre
C_j['T4'] = C_j['vara_str'].map(
    df_tjba_fil[df_tjba_fil['vm_dt_movimento'].dt.quarter == 4]
    .groupby('vara_str')['pje_vl_peso_processual'].sum()
).fillna(0)

C_j

,vara_str,vm_vara,T1,T2,T3,T4
0,VARA_11534,2ª VARA DE FEITOS DE REL DE CONS. CÍVEL E COMERCIAIS,19536,17380,22780,16160
1,VARA_11453,2ª VARA DA FAZENDA PÚBLICA,39696,76804,122372,104922
2,VARA_11369,1ª VARA DE FEITOS DE REL DE CONS. CÍVEL E COMERCIAIS,952,6492,12204,8690
3,VARA_11483,2ª VARA DE RELACOES DE CONSUMO,12776,15414,12762,10386
4,VARA_1124,VARA CÍVEL,3992,6240,7190,8080
...,...,...,...,...,...,...
845,VARA_111163,VARA JURISDIÇÃO PLENA,4,0,0,0
846,VARA_111123,VARA JURISDIÇÃO PLENA,0,0,0,8
847,VARA_11944,VARA RECESSO CIVEL DE GUANAMBI,4,0,0,0
848,VARA_11937,VARA RECESSO CIVEL DE ALAGOINHAS,0,0,0,8


In [16]:
# Criando df com demanda por vara e competencia (D_jk)
D_jk = df_tjba_fil[['vara_str']].drop_duplicates().reset_index(drop=True)
vara = df_tjba_fil[['vm_competencia']].drop_duplicates().reset_index(drop=True)
D_jk = D_jk.merge(vara, how='cross')

# Criar lookup table
lookup_q1 = df_tjba_fil[
    df_tjba_fil['vm_dt_movimento'].dt.quarter == 1
].set_index(['vara_str', 'vm_competencia'])['pje_vl_peso_processual'].groupby(level=[0,1]).sum()

lookup_q2 = df_tjba_fil[
    df_tjba_fil['vm_dt_movimento'].dt.quarter == 2
].set_index(['vara_str', 'vm_competencia'])['pje_vl_peso_processual'].groupby(level=[0,1]).sum()

lookup_q3 = df_tjba_fil[
    df_tjba_fil['vm_dt_movimento'].dt.quarter == 3
].set_index(['vara_str', 'vm_competencia'])['pje_vl_peso_processual'].groupby(level=[0,1]).sum()

lookup_q4 = df_tjba_fil[
    df_tjba_fil['vm_dt_movimento'].dt.quarter == 4
].set_index(['vara_str', 'vm_competencia'])['pje_vl_peso_processual'].groupby(level=[0,1]).sum()

# Transform usando map no índice combinado
# Criando produtivedade ponderada para 1º trimestre
D_jk['T1'] = pd.MultiIndex.from_arrays([D_jk['vara_str'], D_jk['vm_competencia']]).map(lookup_q1).fillna(0)
D_jk['T2'] = pd.MultiIndex.from_arrays([D_jk['vara_str'], D_jk['vm_competencia']]).map(lookup_q2).fillna(0)
D_jk['T3'] = pd.MultiIndex.from_arrays([D_jk['vara_str'], D_jk['vm_competencia']]).map(lookup_q3).fillna(0)
D_jk['T4'] = pd.MultiIndex.from_arrays([D_jk['vara_str'], D_jk['vm_competencia']]).map(lookup_q4).fillna(0)

D_jk

,vara_str,vm_competencia,T1,T2,T3,T4
0,VARA_11534,FAZENDA PUBLICA ATOS ADMINISTRATIVOS,4428,4188,5924,5364
1,VARA_11534,FAZENDA PUBLICA TRIBUTOS,9008,7584,7956,3724
2,VARA_11534,FAMILIA,0,0,0,0
3,VARA_11534,RELACOES DE CONSUMO,676,704,1054,742
4,VARA_11534,ACIDENTES DO TRABALHO,172,232,444,368
...,...,...,...,...,...,...
86695,VARA_111081,EXECUCOES RECESSO SALVADOR,0,0,0,0
86696,VARA_111081,[CEJUSC PREPROCESSUAL] ZONA IV,0,0,0,0
86697,VARA_111081,[CEJUSC PREPROCESSUAL FAMILIA] ZONA 18,0,0,0,0
86698,VARA_111081,CRIMINAL RECESSO SALVADOR,0,0,0,0
